In [108]:
import pandas as pd
import numpy as np

# Data pre-processing
import nltk
from nltk.stem import WordNetLemmatizer

# List of stop words used for data pre-processing
from stop_words_list import stop_words_list
from nltk.corpus import stopwords

# Sentiment Analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Topics modelling
from gsdmm.gsdmm.mgp import MovieGroupProcess

In [109]:
# List of available creators: "MKBHD", "Jonathan Morrison", "Unbox Therapy", "Chris Stuckman", "Jeremy Jahns", "Channel Awesome", "James Charles", "NikkieTutorials", "sophdoeslife"

creator = "NikkieTutorials"
output_df = pd.read_excel(f"../comments_spreadsheets/extracted_comments_{creator}.xlsx")

In [ ]:
duplicates = output_df[output_df.duplicated(("Comments"))]
print ("Count of duplicate comments in dataframe"
, duplicates.shape[0])

print ("Count of unique comments in dataframe"
, output_df.shape[0] - duplicates.shape[0])

# Remove duplicated comments from dataset
unique_df = output_df.drop_duplicates(subset=["Comments"], keep='first', ignore_index=True)
df = unique_df

# Removes line return "\n"
df = df.replace(r'\n',' ', regex=True)

display(df)

# Sentiment Analysis

In [ ]:
sentimentAnalyser = SentimentIntensityAnalyzer()
sentimentScoreList = []
sentimentLabelList = []

for i in df["Comments"].values.tolist():
    sentimentScore = sentimentAnalyser.polarity_scores(i)

    if sentimentScore['compound'] >= 0.05:
        sentimentScoreList.append(sentimentScore['compound'])
        sentimentLabelList.append('Positive')
    elif sentimentScore['compound'] > -0.05 and sentimentScore['compound'] < 0.05:
        sentimentScoreList.append(sentimentScore['compound'])
        sentimentLabelList.append('Neutral')
    elif sentimentScore['compound'] <= -0.05:
        sentimentScoreList.append(sentimentScore['compound'])
        sentimentLabelList.append('Negative')

df["Sentiment"] = sentimentLabelList
df["Sentiment Score"] = sentimentScoreList

display(df.head(10))

In [ ]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

# Convert case text as lowercase, remove punctuation, remove extra whitespace in string and on both sides of string
# Lowercase all the letters
df['lower'] = df['Comments'].str.lower()

# Remove punctuations
df['punctuation_removed'] = df['lower'].str.replace("'", '', regex=True).str.replace('[^\w\s]', ' ', regex=True)

# Remove numbers
df['numbers_removed'] = df['punctuation_removed'].str.replace(" \d+", " ", regex=True)

# Remove extra whitespace
df['extra_spaces_removed'] = df['numbers_removed'].str.replace(' +', ' ', regex=True).str.strip()

# Tokenise
df['tokenised'] = df.apply(lambda row: nltk.word_tokenize(row['extra_spaces_removed']), axis=1)

# Stop words removal
# Initiate stopwords from nltk
stop_words = stopwords.words('english')

# Add additional missing terms
stop_words.extend(stop_words_list)

# Remove stopwords
df['removed_stopwords'] = df['tokenised'].apply(lambda x: [item for item in x if item not in stop_words])

docs = []

for token in df.removed_stopwords:
    docs.append(token)

display(df.head(10))

In [ ]:
mgp = MovieGroupProcess(K=3, alpha=0.1, beta=1, n_iters=30)

vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)

mgp_output = mgp.fit(docs, n_terms)

In [ ]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

In [ ]:
top_index = doc_count.argsort()[-15:][::-1]
print('Indices of most important clusters (by number of docs inside):', top_index)

In [116]:
# Returns the top words present in each topic cluster.
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print('-'*120)

In [ ]:
# Show the top 7 words in term frequency for each cluster
top_words(mgp.cluster_word_distribution, top_index, 7)

In [118]:
topic_dict = {}
topic_names = ['Topic #1', 'Topic #2', 'Topic #3']
for i, topic_num in enumerate(top_index):
    topic_dict[topic_num]=topic_names[i]

In [119]:
# index names
docnames = ['Doc' + str(i) for i in range(len(df))]

# Create a dataframe with the original text and its assigned topic.
def create_topics_dataframe(data_text=df.Comments,  mgp=mgp, threshold=0.3, topic_dict=topic_dict, stem_text=docs):
    result = pd.DataFrame(columns=['topic'])
    for i, text in enumerate(data_text):
        prob = mgp.choose_best_label(stem_text[i])
        if prob[1] >= threshold:
            result.at[i, 'topic'] = topic_dict[prob[0]]
        else:
            result.at[i, 'topic'] = 'Other'
    return result

In [ ]:
dfx = create_topics_dataframe(data_text=df.Comments,  mgp=mgp, threshold=0.3, topic_dict=topic_dict, stem_text=docs)
dfx.topic.value_counts(dropna=False)

In [ ]:
df = pd.merge(df, dfx, left_index = True, right_index = True, how = 'outer')
display(df)

In [ ]:
results = df.groupby(['topic', 'Sentiment']).count().reset_index()

graph_results = results[['topic', 'Sentiment', 'Sentiment Score']]
graph_results = graph_results.pivot(index='topic', columns='Sentiment', values='Sentiment Score').reset_index()

graph_results.set_index('topic', inplace=True)

display(graph_results)

In [ ]:
fig = graph_results.plot.bar(rot=90, figsize=(10,10))

# Uncomment to save the figure as a png to current directory
# fig.figure.savefig(f'{creator}_absa_gsdmm.png' , bbox_inches='tight')